# Rock Paper Scissors Image Classification - Final Submission Dicoding Indonesia
### Course : Belajar Machine Learning Pemula
### Date : 24 April 2021

In [ ]:
# import dependencies
import os
import sys
import time
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.optimizers import Adam

## Preprocessing Function

In [ ]:
# transform image into array
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = img/255
    return img


# transform single image into array for prediction
def single_preprocess_image(image_path):
    img = load_img(image_path, target_size=(224, 224, 3))
    img = img_to_array(img)
    img = img/255
    img = np.expand_dims(img, axis=0)
    return img

In [ ]:
# read dan preprocess training data
def train_data(train_data_path):
    print("\n-- PREPARE TRAINING DATA --")
    train_image = []
    train_label = []

    list_training = list(os.listdir(train_data_path))
    label_size = len(list_training)

    # CARA 2
    # load image from each subject
    sub_num = 0
    for sub in (sorted(list_training)):
        for photo in tqdm(os.listdir(f"{train_data_path}/{sub}"), desc=f"{sub}"):
            if photo.endswith(".jpg"):
                filename = f"{train_data_path}/{sub}/{photo}"
                image_out = preprocess_image(filename)

                # iamge feature and class in binary
                train_image.append(image_out)
                train_label.append(sub)
        if(sub_num == 0):
            np_train = np.array(train_image)
        else:
            np_train = np.concatenate((np_train, np.array(train_image)))
        train_image.clear()
        sub_num += 1

    # encode train label
    le_train = LabelEncoder()
    train_label = le_train.fit_transform(train_label)
    train_label = to_categorical(train_label, label_size)

    # split data
    X_train = np_train
    y_train = train_label

    return (X_train, y_train, le_train)


# read dan preprocess testing data
def test_data(test_data_path):
    print("\n-- PREPARE TESTING DATA --")
    test_image = []
    test_label = []

    list_testing = list(os.listdir(test_data_path))
    label_size = len(list_testing)

    # CARA 2
    # load image from each subject
    sub_num = 0
    for sub in (sorted(list_testing)):
        for photo in tqdm(os.listdir(f"{test_data_path}/{sub}"), desc=f"{sub}"):
            if photo.endswith(".jpg"):
                filename = f"{test_data_path}/{sub}/{photo}"
                image_out = preprocess_image(filename)

                # iamge feature and class in binary
                test_image.append(image_out)
                test_label.append(sub)
        if(sub_num == 0):
            np_test = np.array(test_image)
        else:
            np_test = np.concatenate((np_test, np.array(test_image)))
        test_image.clear()
        sub_num += 1

    # Encode test label
    le_test = LabelEncoder()
    test_label = le_test.fit_transform(test_label)
    test_label = to_categorical(test_label, label_size)

    # split data
    X_test = np_test
    y_test = test_label

    return (X_test, y_test, le_test)

In [ ]:
# Initiate Constant
BS = 20
EPOCHS = 150
MODEL_NAME = "RESNET50"
LEARNING_RATE = 0.001

print("[INFO] Hyperparameter:")
print("Epoch: " + str(EPOCHS))
print("Learning rate: " + str(LEARNING_RATE))
print("Batch Size: " + str(BS))
print("\n")

In [ ]:
# Directory of train and test dataset
training_dataset = f"../Dataset/facescrub-crop-mtcnn-training"
list_training = list(os.listdir(training_dataset))
testing_dataset = f"../Dataset/facescrub-crop-mtcnn-testing"
list_testing = list(os.listdir(testing_dataset))

label_size = len(list_training)
image_per_label = len(os.listdir(f"{training_dataset}/{list_training[0]}"))

In [ ]:
# READ TRAIN AND TEST DATA
X_train, y_train, le_train = train_data(training_dataset)
X_test, y_test, le_test = test_data(testing_dataset)

## Build and Train Model

In [ ]:
# DEFINE MODEL
model = resnet50(label_size)

# SUMMARY MODEL
model.summary()

In [ ]:
# COMPILE MODEL
adam = Adam(learning_rate=LEARNING_RATE)
model.compile(optimizer=adam, loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Initiate Callbacks
my_callbacks = my_callbacks(MODEL_NAME, label_size,
                            image_per_label, EPOCHS, BS)

In [ ]:
# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [ ]:
# Initiate start time
start_time = time.time()

# TRAIN MODEL
Historia = model.fit(
    aug.flow(X_train, y_train, batch_size=BS),
    # X_train, y_train,
    validation_data=(X_test, y_test),
    callbacks=my_callbacks,
    epochs=EPOCHS,
    batch_size=BS)

# end time
print("\n\n--- %s seconds ---" % (time.time() - start_time))

## Evaluate Model

In [ ]:
# EVALUATE MODEL
model = load_my_model(MODEL_NAME, label_size, image_per_label, EPOCHS, BS)

results = model.evaluate(X_test, y_test, batch_size=BS)
print(results)

In [ ]:
# model prediction for testing data
predictions = [np.argmax(x) for x in model.predict(X_test, batch_size=BS)]
tes = [np.argmax(y) for y in y_test]

In [ ]:
# Print classification report
print_class_reports(tes, predictions, le_test.classes_, MODEL_NAME,
                    label_size, image_per_label, EPOCHS, BS)

In [ ]:
# print confusion matrix
print_conf_matrix(tes, predictions, le_test.classes_, MODEL_NAME,
                  label_size, image_per_label, EPOCHS, BS)

## Observe Model Performance using Visualization

In [ ]:
# accuracy
visualize_accuracy(Historia, MODEL_NAME,
                   label_size, image_per_label, EPOCHS, BS)
# loss
visualize_loss(Historia, MODEL_NAME,
               label_size, image_per_label, EPOCHS, BS)